In [ ]:
from rest.base import OdooAPIKey, ProductClient, OdooClient, OdooAPIBase
import os
# 设置环境变量
os.environ['ODOO_ACCESS_KEY'] = 'odoo-api-prod.json'
os.environ['ODOO_ACCESS_KEY_INDEX'] = "0"

api_key = OdooAPIKey.prod()


In [ ]:
# cli = ProductClient(api_key)
# ids = cli.fetch_product_ids()

base = OdooAPIBase(api_key)
cli = base.client


### **1.1 总销售额（Revenue/Sales Volume）**
- **含义**：
  - 统计每个月的销售总额，即所有销售订单的总金额。
  - 公式：`总销售额 = 每个产品的售价 × 销售数量（扣除退款或退货金额）`。
- **用途**：
  - 衡量整体销售表现。
  - 分析季节性趋势或特定促销活动对销售的影响。

In [ ]:
# Get Order Details
domain = [('user_id', 'not in', [8, 6])]
order_ids = cli.search('sale.order', [domain])
orders_ = cli.read('sale.order', [order_ids])
print(orders_[0])


In [ ]:
orders_[100]

In [ ]:
# 获取所有orderline的id
order_line_ids = set()
for od in orders_:
    order_line_ids.update(od['order_line'])

order_line_ids = list(order_line_ids)

print(f"Length: {len(order_line_ids)}")
print(order_line_ids[:10])

In [ ]:
orderlines_ = cli.read('sale.order.line', [order_line_ids])

In [ ]:
orderlines_[50]

In [ ]:
from typing import List, Union
from pydantic import BaseModel

# class SalesOrder(BaseModel):
#     id: int
#     name: str
#     company: str
#     partner: str 
#     state: str
#     date_order: str 
#     invoice_status:str 
#     shipping_weight: float
#     orderline_ids: List[int]

sale_orders = []

for od in orders_:
    so = dict(
        id = od['id'],
        name = od['name'],
        company = od['company_id'][1],
        partner = od['partner_id'][1],
        state = od['state'],
        date_order = od['date_order'],
        invoice_status = od['invoice_status'],
        shipping_weight = od['shipping_weight'],
        orderline_ids = od['order_line']
    )
    sale_orders.append(so)

# print(f"Number of orders: {len(sale_orders)}")

In [ ]:
import pandas as pd
df_sale_order = pd.DataFrame.from_dict(sale_orders)
df_sale_order.to_excel("data/sales_order.xlsx", index=False)


In [ ]:
orderlines = []

for odl in orderlines_:    
    if odl['display_type'] == 'line_note':
        continue
    try:
        line = {
            "order_number": odl['order_id'][1],  # 订单号
            "product_name": odl['name'],  
            "currency": odl['currency_id'][1], 
            "order_partner": odl['order_partner_id'][1], # 客户
            "salesman": odl['salesman_id'][1], # 销售员
            'state': odl['state'], 
            'uom': odl['product_uom'][1],  # 单位
            'product_uom_qty': odl['product_uom_qty'],   #product_qty
            'product_qty': odl['product_qty'],  # 数量
            'price_unit': odl['price_unit'],  # 单价
            'price_subtotal': odl['price_subtotal'], # 小计
            'price_tax': odl['price_tax'], # 含税
            'price_total': odl['price_total'], # 总计
            'qty_to_invoice': odl['qty_to_invoice'], 
            'qty_to_deliver': odl['qty_to_deliver'],
            'product_type': odl['product_type'],
            'create_date': odl['create_date'],
            'is_delivery': odl['is_delivery'],
        }
    except Exception as e:
        print(e)
        print(odl)
    orderlines.append(line)
df_sale_order_lines = pd.DataFrame.from_dict(orderlines)
df_sale_order_lines.to_excel('data/sales_order_lines.xlsx')



In [ ]:
import datetime
# 数据清洗

# 被取消的orderlines
df_filtered_orderlines = df_sale_order_lines[df_sale_order_lines['state']== 'sale']
# product_type == product
df_filtered_orderlines = df_filtered_orderlines[df_filtered_orderlines['product_type']== 'product']
# qty_to_invoice == 0 and qty_to_deliver == 0
qty_to_invoice_zero = df_filtered_orderlines['qty_to_invoice'] == 0
qty_to_deliver_zero = df_filtered_orderlines['qty_to_deliver'] == 0
df_filtered_orderlines = df_filtered_orderlines[qty_to_invoice_zero & qty_to_deliver_zero]

# price_unit == 0
df_filtered_orderlines = df_filtered_orderlines[df_filtered_orderlines['price_unit']!= 0]

# Create month, year columns, 2024-07-25 10:46:01 to 2024-07
df_filtered_orderlines['year'] = pd.to_datetime(df_filtered_orderlines['create_date']).dt.strftime('%Y')
df_filtered_orderlines['month'] = pd.to_datetime(df_filtered_orderlines['create_date']).dt.strftime('%Y-%m')

len(df_filtered_orderlines)
df_filtered_orderlines.to_excel('data/sales_order_lines.xlsx')

In [ ]:
odoo_orderlines = df_filtered_orderlines.copy()
# 按订单号聚合订单明细.
df_agg_orderlines = odoo_orderlines.groupby('order_number')  \
    .agg({'price_subtotal': 'sum', 'create_date': 'first', 'price_unit': 'sum', 
          'order_partner': 'first', 'salesman': 'first', 
          'price_tax': 'sum'}) \
    .reset_index()

df_agg_orderlines.head()

In [ ]:
# 统计每一个客户的销售额 (不含运费，不含为结的)
df_customer_sales = odoo_orderlines.groupby('order_partner') \
                        .agg({'price_subtotal': sum, 'currency': 'first'})
df_customer_sales = df_customer_sales.sort_values('price_subtotal', ascending=False)
df_customer_sales_top10 = df_customer_sales.head(10)
df_customer_sales_bottom10 = df_customer_sales.tail(10)
customer_sort_by_sales = df_customer_sales.reset_index()['order_partner'].tolist()
df_customer_sales_top10


In [ ]:
# 统计每一个产品的销售额 (不含运费，不含为结的)
df_product_sales = odoo_orderlines.groupby(['product_name']) \
                        .agg({'price_subtotal': sum, 
                              'currency': 'first', 
                              'product_uom_qty': sum,
                              # 'product_qty': sum,
                              'uom': 'first'})
df_product_sales = df_product_sales.sort_values('price_subtotal', ascending=False)
df_product_sales_top10 = df_product_sales.head(10)
df_product_sales_bottom10 = df_product_sales.tail(10)
product_sort_by_sales = df_product_sales.reset_index()['product_name'].tolist()
df_product_sales

In [ ]:
# 统计每一个客户每个月的销售额 (不含运费，不含为结的)
df_customer_month_sales = odoo_orderlines.groupby(['order_partner', 'month', ]) \
                            .agg({'price_subtotal': sum, 
                                  'currency': 'first', 
                                  'order_number': 'nunique'})
df_customer_month_sales = df_customer_month_sales.rename(columns={'order_number': 'order_count'})
df_customer_month_sales['avg_price'] = df_customer_month_sales['price_subtotal'] / df_customer_month_sales['order_count']
df_customer_month_sales['avg_price'] = df_customer_month_sales['avg_price'].round(2)
df_customer_month_sales.to_excel('data/customer_month_sales.xlsx')
# sort by customer_sort_by_sales
# 将客户名列表转为分类类型，以指定排序顺序
df_customer_month_sales['customer_sort'] = pd.Categorical(
    df_customer_month_sales.index.get_level_values('order_partner'),  # 获取 MultiIndex 中的 'order_partner'
    categories=customer_sort_by_sales,  # 自定义排序规则
    ordered=True               # 指定为有序
)

df_customer_month_sales = df_customer_month_sales.sort_values(by='customer_sort')
df_customer_month_sales = df_customer_month_sales.drop(columns=['customer_sort'])

df_customer_month_sales.to_excel('data/customer_month_sales.xlsx')
df_customer_month_sales.head(10)



In [ ]:
# 统计每个产品每个月卖出的数量和金额
df_product_month_sales = odoo_orderlines.groupby(['product_name', 'month', ]) \
                            .agg({'price_subtotal': sum, 
                                  'currency': 'first', 
                                  'order_number': 'nunique'})
df_product_month_sales

In [ ]:
# 统计客户买了什么哪些产品

In [ ]:
# Save in sheets
with pd.ExcelWriter('data/sales.xlsx') as writer:
    df_product_month_sales.to_excel(writer, sheet_name='product_month_sales')
    df_product_sales.to_excel(writer, sheet_name='product_sales')
    df_customer_month_sales.to_excel(writer, sheet_name='customer_month_sales')
    # df_customer_sales_top10.to_excel(writer, sheet_name='customer_sales_top10')
    # df_customer_sales_bottom10.to_excel(writer, sheet_name='customer_sales_bottom10')

In [ ]:
# Step 1: 按订单计算销售额
order_sales = odoo_orderlines.groupby(['order_partner', 'month', 'order_number']) \
    .agg({'price_subtotal': 'sum'}).reset_index()

# Step 2: 按客户和月份汇总计算总销售额与订单数量
customer_month_sales = order_sales.groupby(['order_partner', 'month']) \
    .agg({
        'price_subtotal': 'sum',  # 总销售额
        'order_number': 'count'  # 订单数量
    }).rename(columns={
        'price_subtotal': 'total_sales', 
        'order_number': 'order_count'
    }).reset_index()

# Step 3: 计算平均每个订单的销售额
customer_month_sales['avg_order_sales'] = customer_month_sales['total_sales'] / customer_month_sales['order_count']

customer_month_sales


In [ ]:
# 1. 总销售额 (不含税) 和 含税总销售额
sales_metrics = odoo_orderlines.groupby('salesman').agg({
    'price_subtotal': 'sum',  # 不含税总额
    'price_total': 'sum',     # 含税总额
    'order_number': 'nunique',  # 订单数
    'product_uom_qty': 'sum',  # 总销售数量
    'qty_to_deliver': 'sum',  # 未交付数量
    'qty_to_invoice': 'sum',  # 未开票数量
}).rename(columns={
    'price_subtotal': 'total_sales',
    'price_total': 'total_sales_taxed',
    'order_number': 'total_orders',
    'product_uom_qty': 'total_qty_sold',
    'qty_to_deliver': 'total_qty_to_deliver',
    'qty_to_invoice': 'total_qty_to_invoice',
})

# 2. 平均订单金额
sales_metrics['avg_order_value'] = sales_metrics['total_sales'] / sales_metrics['total_orders']

# 3. 最畅销商品 (按数量和金额)
top_products = df_sale_order_lines.groupby('product_name').agg({
    'product_uom_qty': 'sum',
    'price_subtotal': 'sum'
}).sort_values(by='product_uom_qty', ascending=False)

# 4. 客户贡献 (按客户总销售额)
customer_contribution = df_sale_order_lines.groupby('order_partner').agg({
    'price_subtotal': 'sum'
}).sort_values(by='price_subtotal', ascending=False)


# 保存结果为 Excel 文件或直接查看
sales_metrics.to_excel('data/sales_metrics.xlsx')
top_products.to_excel('data/top_products.xlsx')
customer_contribution.to_excel('data/customer_contribution.xlsx')

print("销售指标表已保存为 'sales_metrics.xlsx'")
print("最畅销商品表已保存为 'top_products.xlsx'")
print("客户贡献表已保存为 'customer_contribution.xlsx'")